In [8]:
import pandas as pd
df = pd.read_csv (r'D:\archive\PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [9]:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import load_boston
    from sklearn.metrics import mean_squared_error
    from sklearn.linear_model import LogisticRegression

In [2]:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import load_boston
    from sklearn.metrics import mean_squared_error
    from sklearn.linear_model import LogisticRegression
    
#isFraud is the output so we drop it from df_X
   
    df_X=df.drop('isFraud',axis=1)
    
#nameOrig and nameDest columns are strings containing unique values so it is better to be dropped
    
    df_X.drop('nameOrig',axis=1,inplace=True)
    df_X.drop('nameDest',axis=1,inplace=True)
    
    df_Y=df['isFraud']
#type column  contains strings and dont have unique values
#So we can not drop it and we have to transform all strings using this encoder

    from sklearn.preprocessing import OrdinalEncoder

    ord_enc = OrdinalEncoder()
    df_X["type"] = ord_enc.fit_transform(df_X[["type"]])
    
    
#rms_array is the array tht contains error for each degree so that
#we can know the best degree by getting the lowest error in the array
    rms_array=[]
    
#for loop loops on different degrees from 1 to 8
#LogisticRegression is used in this assignment
    for i in range(1,2):
      poly = PolynomialFeatures(degree= i, include_bias=False)
      poly_features = poly.fit_transform(df_X)
        
#Divide dataset into Train/Cross Validation/Test        
      X_train, X_test, y_train, y_test = train_test_split(poly_features, df_Y, test_size=0.4, random_state=1)
      X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
        
      poly_reg_model = LogisticRegression()
      poly_reg_model.fit(X_train, y_train)
      poly_reg_y_predicted = poly_reg_model.predict(X_val)
      Reg_err = np.sqrt(mean_squared_error(y_val, poly_reg_y_predicted))
      rms_array.append(Reg_err)

In [3]:
print(rms_array)


[0.04243989645857028]


In [ ]:
#Test on degree with the lowest error from rms_array to make sure that no big difference between them 
  poly = PolynomialFeatures(degree= 2, include_bias=False)
  poly_features = poly.fit_transform(df_X)
  X_train, X_test, y_train, y_test = train_test_split(poly_features, df_Y, test_size=0.4, random_state=1)
  X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
  poly_reg_model = LogisticRegression()
  poly_reg_model.fit(X_train, y_train)
  poly_reg_y_predicted = poly_reg_model.predict(X_test)
  Reg_err = np.sqrt(mean_squared_error(y_test, poly_reg_y_predicted))
  print(Reg_err)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev

#Apply StratifiedKFold technique
#split data into 4
#shuffle is true to randomly pick data from all the data set
#LogisticRegression is used 
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)
lst_accu_stratified = []

for train_index, test_index in skf.split(poly_features, df_Y):
    x_train_fold, x_test_fold = poly_features[train_index], poly_features[test_index]
    y_train_fold, y_test_fold = df_Y[train_index], df_Y[test_index]
    poly_reg_model = LogisticRegression()
    poly_reg_model.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(poly_reg_model.score(x_test_fold, y_test_fold))
  
##################################################################################

print('Possible accuracy:', lst_accu_stratified)

print('\nAccuracy:',
      mean(lst_accu_stratified)*100, '%')

print('\n Max_Accuracy :',
      max(lst_accu_stratified)*100, '%')

print('\nMin_Accuracy:',
      min(lst_accu_stratified)*100, '%')

print('\nStandard_Deviation is:', stdev(lst_accu_stratified))

In [ ]:
#Applying Regularization
#Theta is deacreased near to zero but not equal to zero
#alpha is the intensity of the penalty

from sklearn.linear_model import LogisticRegression 
alpha_arr=[0.1,0.01,1]
result=[]
df_X=df.drop('isFraud',axis=1)
df_X.drop('nameOrig',axis=1,inplace=True)
df_X.drop('nameDest',axis=1,inplace=True)
df_Y=df['isFraud']

from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
df_X["type"] = ord_enc.fit_transform(df_X[["type"]])
poly = PolynomialFeatures(degree= 2, include_bias=False)
poly_features = poly.fit_transform(df_X)
X_train, X_test, y_train, y_test = train_test_split(poly_features, df_Y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
  
poly_reg_model = LogisticRegression(penalty='l1', C=1 ,solver='saga', max_iter=2)
poly_reg_model.fit(X_train, y_train)
poly_reg_y_predicted = poly_reg_model.predict(X_test)
Reg_err = np.sqrt(mean_squared_error(y_test, poly_reg_y_predicted))
result.append(Reg_err)

In [11]:
print(result)

[0.03553652418932332]
